# imports

In [1]:
# --- Setup
import os, sys, importlib, json, time, traceback

# (optional) .env
try:
    from dotenv import load_dotenv
    load_dotenv()
except Exception:
    pass

# Logging + API key
os.environ.setdefault("ORS_LOG_LEVEL", "INFO")
if not os.getenv("ORS_API_KEY"):
    raise SystemExit("Set ORS_API_KEY in your environment or .env before running this notebook.")

# Ensure project root on sys.path (this notebook sits in /tests)
ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if os.path.basename(os.getcwd()) == "tests":
    ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.insert(0, ROOT)

# Imports from refactored modules
from modules.road.ors_client import ORSConfig, ORSClient
from modules.addressing import resolver

# Optional cabotage bits (skip gracefully if not present)
try:
    from modules.cabotage.ports_index import load_cts
    from modules.cabotage.ports_nearest import find_nearest_port
except Exception:
    load_cts = None
    find_nearest_port = None

# Hot reload while iterating
importlib.reload(resolver)

# ORS client (default profile = driving-hgv)
cfg = ORSConfig(default_country="BR")
ors = ORSClient(cfg)

# Pretty printer
def jprint(obj) -> None:
    print(json.dumps(obj, ensure_ascii=False, indent=2, sort_keys=True))

# Soft assertion helper
def _assert_ok(cond: bool, msg: str):
    if not cond:
        print("⚠️  ASSERTION WARN:", msg)


### 1) Addressing — `resolve_point` smoke tests

In [2]:
def _check_point(tag: str, p: dict):
    _assert_ok(isinstance(p, dict), f"{tag}: not a dict")
    _assert_ok("lat" in p and "lon" in p, f"{tag}: missing lat/lon")
    _assert_ok(isinstance(p["lat"], (int, float)) and isinstance(p["lon"], (int, float)), f"{tag}: lat/lon not numeric")

tests = []

# a) tuple
tests.append(("tuple", (-23.55052, -46.63331)))

# b) dict lat/lon
tests.append(("dict-latlon", {"lat": -23.9608, "lon": -46.3336, "label": "Santos Ponta da Praia"}))

# c) structured dict
tests.append((
      "structured"
    , {
          "street": "Avenida Paulista"
        , "housenumber": "1578"
        , "locality": "São Paulo"
        , "region": "SP"
        , "postalcode": "01310-200"
        , "country": "BR"
      }
))

# d) lat,lon string
tests.append(("latlon-str", "-23.55052,-46.63331"))

# e) CEP with hyphen
tests.append(("cep-hyph", "01310-200"))

# f) CEP digits only
tests.append(("cep-digits", "11010091"))

# g) free text
tests.append(("freetext", "Porto de Santos, Santos, SP"))

results = {}
for label, value in tests:
    print("\n---", label, "---")
    t0 = time.time()
    p = resolver.resolve_point(value, ors=ors)
    dt = (time.time() - t0) * 1000
    _check_point(label, p)
    print(f"ok in {dt:.0f} ms →", p)
    results[label] = p

# Negative cases (should raise ValueError/TypeError but keep notebook alive)
neg = [
      ("bad-cep", "01310-20")
    , ("bad-type", {"foo": "bar"})
]
for label, value in neg:
    print("\n---", label, "(expected error) ---")
    try:
        resolver.resolve_point(value, ors=ors)
        print("⚠️  expected an exception but got none")
    except Exception as e:
        print("ok →", type(e).__name__, str(e)[:120])


[17:12:57] INFO cabosupernet.road.ors_client | GEOCODE structured params={"address": "1578 Avenida Paulista", "country": "BR", "locality": "São Paulo", "postalcode": "01310-200", "region": "SP", "size": 1}
[17:12:57] INFO cabosupernet.road.ors_client | GEOCODE structured params={"country": "BR", "postalcode": "01310200", "size": 1}
[17:12:57] INFO cabosupernet.road.ors_client | GEOCODE structured params={"country": "BR", "postalcode": "01310-200", "size": 1}
[17:12:57] INFO cabosupernet.road.ors_client | GEOCODE text='01310-200' country=BR size=1



--- tuple ---
ok in 0 ms → {'lat': -23.55052, 'lon': -46.63331, 'label': '-23.550520,-46.633310'}

--- dict-latlon ---
ok in 0 ms → {'lat': -23.9608, 'lon': -46.3336, 'label': 'Santos Ponta da Praia'}

--- structured ---
ok in 2 ms → {'lat': -23.555036, 'lon': -46.663479, 'label': 'Avenida Paulista, São Paulo, Brazil'}

--- latlon-str ---
ok in 0 ms → {'lat': -23.55052, 'lon': -46.63331, 'label': '-23.550520,-46.633310'}

--- cep-hyph ---


[17:12:58] INFO cabosupernet.road.ors_client | GEOCODE text='Avenida Paulista, Bela Vista, São Paulo, SP' country=BR size=1
[17:12:58] INFO cabosupernet.road.ors_client | GEOCODE structured params={"country": "BR", "postalcode": "11010091", "size": 1}
[17:12:58] INFO cabosupernet.road.ors_client | GEOCODE structured params={"country": "BR", "postalcode": "11010-091", "size": 1}
[17:12:58] INFO cabosupernet.road.ors_client | GEOCODE text='11010-091' country=BR size=1


ok in 1362 ms → {'lat': -23.563498, 'lon': -46.653985, 'label': 'Ciclovia da Avenida Paulista, São Paulo, Brazil'}

--- cep-digits ---


[17:12:59] INFO cabosupernet.road.ors_client | GEOCODE text='Rua Frei Gaspar, Centro, Santos, SP' country=BR size=1
[17:12:59] INFO cabosupernet.road.ors_client | GEOCODE text='Porto de Santos, Santos, SP' country=BR size=1
[17:12:59] INFO cabosupernet.road.ors_client | GEOCODE text='01310-20' country=BR size=1


ok in 661 ms → {'lat': -23.935639, 'lon': -46.325439, 'label': 'Centro, Santos, SP, Brazil'}

--- freetext ---
ok in 1 ms → {'lat': -23.932511, 'lon': -46.324739, 'label': 'Alfandega do Porto de Santos, Santos, SP, Brazil'}

--- bad-cep (expected error) ---
⚠️  expected an exception but got none

--- bad-type (expected error) ---
ok → TypeError Unsupported point type. Use string (address/CEP/city/'lat,lon'), dicts (lat/lon or structured), or (lat,lon) tuple/list.


### 2) Road — HGV (truck) routing smoke tests

In [3]:
def run_case(label: str, origin, destination, **kwargs):
    print("\n" + "=" * 80)
    print(label)
    print("-" * 80)
    print("origin:", origin)
    print("destination:", destination)
    t0 = time.time()
    try:
        resp = ors.route_road(
              origin
            , destination
            , **kwargs
        )
        # Keep lean output (no segments)
        resp.pop("segments", None)
        elapsed = (time.time() - t0) * 1000
        print(f"\n✓ OK in {elapsed:.0f} ms — result:\n")
        jprint(resp)
        # sanity
        _assert_ok(resp.get("distance_m") and resp["distance_m"] > 1000, "distance should be > 1km")
        _assert_ok(resp.get("duration_s") and resp["duration_s"] > 60, "duration should be > 60s")
        return resp
    except Exception as e:
        elapsed = (time.time() - t0) * 1000
        print(f"\n✗ ERROR in {elapsed:.0f} ms — details:\n")
        body = None
        resp_e = getattr(e, "response", None)
        if resp_e is not None:
            try:
                body = resp_e.json()
            except Exception:
                body = resp_e.text
        err_info = {
              "error_type": type(e).__name__
            , "http_status": getattr(resp_e, "status_code", None)
            , "server_body": body
            , "exception_str": str(e)
        }
        jprint(err_info)
        print("\nTraceback (for debugging):")
        traceback.print_exc()
        raise

# --- Cases (truck profile)
r1 = run_case(
      "1) Free-text address"
    , "Av. Paulista, 1578, São Paulo, SP"
    , "Porto de Santos, Santos, SP"
)

r2 = run_case(
      "2) CEP-only (postal code)"
    , "01310-200"
    , "11010-091"
)

r3 = run_case(
      "3) City names"
    , "São Paulo, SP"
    , "Rio de Janeiro, RJ"
)

r4 = run_case(
      "4) Coordinates (tuple) → CEP"
    , (-23.55052, -46.63331)
    , "11010913"
)

origin_struct = {
      "street": "Avenida Paulista"
    , "housenumber": "1578"
    , "locality": "São Paulo"
    , "region": "SP"
    , "postalcode": "01310-200"
    , "country": "BR"
}
r5 = run_case(
      "5) Structured dict → explicit lat/lon"
    , origin_struct
    , { "lat": -23.9608, "lon": -46.3336, "label": "Santos Ponta da Praia" }
)


[17:12:59] INFO cabosupernet.road.ors_client | GEOCODE text='Av. Paulista, 1578, São Paulo, SP' country=BR size=1
[17:12:59] INFO cabosupernet.road.ors_client | GEOCODE text='Porto de Santos, Santos, SP' country=BR size=1
[17:12:59] INFO cabosupernet.road.ors_client | ROUTE try1 driving-hgv coords=[[-46.663479, -23.555036], [-46.324739, -23.932511]]
[17:12:59] INFO cabosupernet.road.ors_client | ROUTE OK km=74.39 hrs=1.47
[17:12:59] INFO cabosupernet.road.ors_client | GEOCODE structured params={"country": "BR", "postalcode": "01310200", "size": 1}
[17:12:59] INFO cabosupernet.road.ors_client | GEOCODE structured params={"country": "BR", "postalcode": "01310-200", "size": 1}
[17:12:59] INFO cabosupernet.road.ors_client | GEOCODE text='01310-200' country=BR size=1



1) Free-text address
--------------------------------------------------------------------------------
origin: Av. Paulista, 1578, São Paulo, SP
destination: Porto de Santos, Santos, SP

✓ OK in 9 ms — result:

{
  "destination": {
    "label": "Alfandega do Porto de Santos, Santos, SP, Brazil",
    "lat": -23.932511,
    "lon": -46.324739
  },
  "distance_m": 74393.0,
  "duration_s": 5303.1,
  "origin": {
    "label": "Avenida Paulista, São Paulo, Brazil",
    "lat": -23.555036,
    "lon": -46.663479
  }
}

2) CEP-only (postal code)
--------------------------------------------------------------------------------
origin: 01310-200
destination: 11010-091


[17:12:59] INFO cabosupernet.road.ors_client | GEOCODE text='Avenida Paulista, Bela Vista, São Paulo, SP' country=BR size=1
[17:12:59] INFO cabosupernet.road.ors_client | GEOCODE structured params={"country": "BR", "postalcode": "11010091", "size": 1}
[17:12:59] INFO cabosupernet.road.ors_client | GEOCODE structured params={"country": "BR", "postalcode": "11010-091", "size": 1}
[17:12:59] INFO cabosupernet.road.ors_client | GEOCODE text='11010-091' country=BR size=1
[17:13:00] INFO cabosupernet.road.ors_client | GEOCODE text='Rua Frei Gaspar, Centro, Santos, SP' country=BR size=1
[17:13:00] INFO cabosupernet.road.ors_client | ROUTE try1 driving-hgv coords=[[-46.653985, -23.563498], [-46.325439, -23.935639]]
[17:13:00] INFO cabosupernet.road.ors_client | ROUTE OK km=72.37 hrs=1.41
[17:13:00] INFO cabosupernet.road.ors_client | GEOCODE text='São Paulo, SP' country=BR size=1
[17:13:00] INFO cabosupernet.road.ors_client | GEOCODE text='Rio de Janeiro, RJ' country=BR size=1
[17:13:00] INFO 


✓ OK in 1360 ms — result:

{
  "destination": {
    "label": "Centro, Santos, SP, Brazil",
    "lat": -23.935639,
    "lon": -46.325439
  },
  "distance_m": 72373.5,
  "duration_s": 5086.6,
  "origin": {
    "label": "Ciclovia da Avenida Paulista, São Paulo, Brazil",
    "lat": -23.563498,
    "lon": -46.653985
  }
}

3) City names
--------------------------------------------------------------------------------
origin: São Paulo, SP
destination: Rio de Janeiro, RJ

✓ OK in 9 ms — result:

{
  "destination": {
    "label": "Rio de Janeiro, Brazil",
    "lat": -22.935024,
    "lon": -43.518246
  },
  "distance_m": 405393.4,
  "duration_s": 23439.5,
  "origin": {
    "label": "São Paulo, Brazil",
    "lat": -23.570533,
    "lon": -46.663713
  }
}

4) Coordinates (tuple) → CEP
--------------------------------------------------------------------------------
origin: (-23.55052, -46.63331)
destination: 11010913


[17:13:01] INFO cabosupernet.road.ors_client | GEOCODE text='Rua Martin Affonso, Centro, Santos, SP' country=BR size=1
[17:13:01] INFO cabosupernet.road.ors_client | ROUTE try1 driving-hgv coords=[[-46.63331, -23.55052], [-46.325439, -23.935639]]
[17:13:01] INFO cabosupernet.road.ors_client | ROUTE OK km=69.81 hrs=1.35
[17:13:01] INFO cabosupernet.road.ors_client | GEOCODE structured params={"address": "1578 Avenida Paulista", "country": "BR", "locality": "São Paulo", "postalcode": "01310-200", "region": "SP", "size": 1}
[17:13:01] INFO cabosupernet.road.ors_client | ROUTE try1 driving-hgv coords=[[-46.663479, -23.555036], [-46.3336, -23.9608]]
[17:13:01] INFO cabosupernet.road.ors_client | ROUTE OK km=77.82 hrs=1.56



✓ OK in 803 ms — result:

{
  "destination": {
    "label": "Centro, Santos, SP, Brazil",
    "lat": -23.935639,
    "lon": -46.325439
  },
  "distance_m": 69812.2,
  "duration_s": 4850.8,
  "origin": {
    "label": "-23.550520,-46.633310",
    "lat": -23.55052,
    "lon": -46.63331
  }
}

5) Structured dict → explicit lat/lon
--------------------------------------------------------------------------------
origin: {'street': 'Avenida Paulista', 'housenumber': '1578', 'locality': 'São Paulo', 'region': 'SP', 'postalcode': '01310-200', 'country': 'BR'}
destination: {'lat': -23.9608, 'lon': -46.3336, 'label': 'Santos Ponta da Praia'}

✓ OK in 6 ms — result:

{
  "destination": {
    "label": "Santos Ponta da Praia",
    "lat": -23.9608,
    "lon": -46.3336
  },
  "distance_m": 77816.9,
  "duration_s": 5601.9,
  "origin": {
    "label": "Avenida Paulista, São Paulo, Brazil",
    "lat": -23.555036,
    "lon": -46.663479
  }
}


In [6]:
r6 = run_case(
      "6) Neighborhood names"
    , "Campolim, Sorocaba, SP"
    , "Copacabana, Rio de Janeiro, RJ"
)

[17:16:06] INFO cabosupernet.road.ors_client | GEOCODE text='Campolim, Sorocaba, SP' country=BR size=1



6) Neighborhood names
--------------------------------------------------------------------------------
origin: Campolim, Sorocaba, SP
destination: Copacabana, Rio de Janeiro, RJ


[17:16:07] INFO cabosupernet.road.ors_client | GEOCODE text='Copacabana, Rio de Janeiro, RJ' country=BR size=1
[17:16:07] INFO cabosupernet.road.ors_client | ROUTE try1 driving-hgv coords=[[-47.468989, -23.520474], [-43.187679, -22.976478]]
[17:16:17] INFO cabosupernet.road.ors_client | ROUTE OK km=539.1 hrs=8.24



✓ OK in 10331 ms — result:

{
  "destination": {
    "label": "Copacabana, Rio de Janeiro, Brazil",
    "lat": -22.976478,
    "lon": -43.187679
  },
  "distance_m": 539100.9,
  "duration_s": 29670.6,
  "origin": {
    "label": "Rua Sylvio Campolim, Sorocaba, SP, Brazil",
    "lat": -23.520474,
    "lon": -47.468989
  }
}
